In [5]:
!ls -lrth ../input/b7ns-final-672-300w-f0-load13-load1-14ep/b7ns_final_672_300w_f0_load13_load1_14ep_fold0_ep4.pth
!ls -lrth ../input/b6ns-final-768-300w-f1-load28-5ep-1e-5/b6ns_final_768_300w_f1_load28_5ep_1e-5_fold1_ep5.pth
!ls -lrth ../input/b5ns-final-768-300w-f2-load16-20ep/b5ns_final_768_300w_f2_load16_20ep_fold2_ep1.pth
!ls -lrth ../input/b4ns-final-768-300w-f0-load16-20ep-load1-20ep/b4ns_final_768_300w_f0_load16_20ep_load1_20ep_fold0_ep4.pth
!ls -lrth ../input/b3ns-final-768-300w-f1-load29-5ep5ep/b3ns_final_768_300w_f1_load29_5ep5ep_fold1_ep5.pth
!ls -lrth ../input/nest101-final-768-300w-f4-load16-19ep-load1-16ep/nest101_final_768_300w_f4_load16_19ep_load1_16ep_fold4_ep5.pth
!ls -lrth ../input/rex20-ddp-final-768-300w-f4-35ep-load20resume/rex20_DDP_final_768_300w_f4_35ep_load20resume_fold4_ep31.pth
!ls -lrth ../input/b6ns-ddp-final-512-300w-f1-40ep/b6ns_DDP_final_512_300w_f1_40ep_fold1_ep36.pth
!ls -lrth ../input/b5ns-final-768-300w-f2-load33-5ep-3e-5-32g/b5ns_final_768_300w_f2_load33_5ep_3e-5_32G_fold2_ep4.pth

ls: cannot access '../input/b7ns-final-672-300w-f0-load13-load1-14ep/b7ns_final_672_300w_f0_load13_load1_14ep_fold0_ep4.pth': No such file or directory
ls: cannot access '../input/b6ns-final-768-300w-f1-load28-5ep-1e-5/b6ns_final_768_300w_f1_load28_5ep_1e-5_fold1_ep5.pth': No such file or directory
ls: cannot access '../input/b5ns-final-768-300w-f2-load16-20ep/b5ns_final_768_300w_f2_load16_20ep_fold2_ep1.pth': No such file or directory
ls: cannot access '../input/b4ns-final-768-300w-f0-load16-20ep-load1-20ep/b4ns_final_768_300w_f0_load16_20ep_load1_20ep_fold0_ep4.pth': No such file or directory
ls: cannot access '../input/b3ns-final-768-300w-f1-load29-5ep5ep/b3ns_final_768_300w_f1_load29_5ep5ep_fold1_ep5.pth': No such file or directory
ls: cannot access '../input/nest101-final-768-300w-f4-load16-19ep-load1-16ep/nest101_final_768_300w_f4_load16_19ep_load1_16ep_fold4_ep5.pth': No such file or directory
ls: cannot access '../input/rex20-ddp-final-768-300w-f4-35ep-load20resume/rex20_DDP_fi

In [6]:
SKIP_COMMIT = False

In [7]:
import sys
# sys.path = [
#     '../input/geffnet-20200820',
#     '../input/rexnetv1',
#     '../input/resnest/ResNeSt-master'
# ] + sys.path
sys.path.append("/mnt/data/sjx/CS498_DL_Project")
print(sys.path)

['/mnt/data/sjx/CS498_DL_Project/Google-Landmark-Recognition-2020-3rd-Place-Solution', '/home/jingxiang/anaconda3/lib/python38.zip', '/home/jingxiang/anaconda3/lib/python3.8', '/home/jingxiang/anaconda3/lib/python3.8/lib-dynload', '', '/home/jingxiang/.local/lib/python3.8/site-packages', '/home/jingxiang/anaconda3/lib/python3.8/site-packages', '/home/jingxiang/anaconda3/lib/python3.8/site-packages/smplx-0.1.21-py3.8.egg', '/home/jingxiang/anaconda3/lib/python3.8/site-packages/torchgeometry-0.1.2-py3.8.egg', '/home/jingxiang/anaconda3/lib/python3.8/site-packages/apex-0.1-py3.8.egg', '/home/jingxiang/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/home/jingxiang/.ipython', '/mnt/data/sjx/CS498_DL_Project', '/mnt/data/sjx/CS498_DL_Project']


In [34]:
import os
import cv2
import glob
import math
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import albumentations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm as tqdm

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

import geffnet

In [28]:
# data_dir = '../input/landmark-recognition-2020/'
# model_dir = '../input/landmarkmodels/'
data_dir = '/mnt/data/sjx/CS498_DL_Project/data/'
model_dir = '/mnt/data/sjx/CS498_DL_Project/Google-Landmark-Recognition-2020-3rd-Place-Solution/weights/'
df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df['filepath'] = df['id'].apply(lambda x: os.path.join(data_dir, 'train', x[0], x[1], x[2], f'{x}.jpg'))
df_demo_0 = df[df['filepath'].str.startswith('/mnt/data/sjx/CS498_DL_Project/data/train/0/0')]
df_demo_1 = df[df['filepath'].str.startswith('/mnt/data/sjx/CS498_DL_Project/data/train/0/1')]
df_demo_2 = df[df['filepath'].str.startswith('/mnt/data/sjx/CS498_DL_Project/data/train/0/2')]
df_demo_3 = df[df['filepath'].str.startswith('/mnt/data/sjx/CS498_DL_Project/data/train/0/3')]

df_demo = df_demo_0.append([df_demo_1, df_demo_2, df_demo_3])
# get train and valid dataset
df = df_demo
df.to_csv('/mnt/data/sjx/CS498_DL_Project/data/tmp/train_demo.csv')


df_sub = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

df_test = df_sub[['id']].copy()
df_test['filepath'] = df_test['id'].apply(lambda x: os.path.join(data_dir, 'test', x[0], x[1], x[2], f'{x}.jpg'))
df_test.to_csv(os.path.join('/mnt/data/sjx/CS498_DL_Project', 'data/test.csv'), index = False)
use_metric = True

In [29]:
device = torch.device('cuda')
batch_size = 4
num_workers = 4
out_dim = 81313 


In [30]:
transforms_672 = albumentations.Compose([
    albumentations.Resize(672, 672),
    albumentations.Normalize()
])

transforms_768 = albumentations.Compose([
    albumentations.Resize(768, 768),
    albumentations.Normalize()
])
transforms_512 = albumentations.Compose([
    albumentations.Resize(512, 512),
    albumentations.Normalize()
])


class LandmarkDataset(Dataset):
    def __init__(self, csv, split, mode, transforms=[transforms_672, transforms_768,transforms_512]):

        self.csv = csv.reset_index()
        self.split = split
        self.mode = mode
        self.transform672 = transforms[0]
        self.transform768 = transforms[1]
        self.transform512 = transforms[2]

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        print(row.filepath)
        image = cv2.imread(row.filepath)
        image = image[:, :, ::-1]
        
        res0 = self.transform672(image=image)
        image0 = res0['image'].astype(np.float32)
        image0 = image0.transpose(2, 0, 1)        

        res1 = self.transform768(image=image)
        image1 = res1['image'].astype(np.float32)
        image1 = image1.transpose(2, 0, 1)    
        
        res3 = self.transform512(image=image)
        image3 = res3['image'].astype(np.float32)        
        image3 = image3.transpose(2, 0, 1)   
               
        
        if self.mode == 'test':
            return torch.tensor(image0), torch.tensor(image1) , torch.tensor(image3)

In [31]:
if df.shape[0] > 100001: # commit
    df = df[df.index % 10 == 0].iloc[500:1000].reset_index(drop=True)
    df_test = df_test.head(101).copy()

dataset_query = LandmarkDataset(df, 'test', 'test')
query_loader = torch.utils.data.DataLoader(dataset_query, batch_size=batch_size, num_workers=num_workers)

dataset_test = LandmarkDataset(df_test, 'test', 'test')
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers)

print(len(dataset_query), len(dataset_test))

24478 10345


In [32]:
dataset_query[0][0].shape, dataset_query[0][1].shape, dataset_query[0][2].shape

/mnt/data/sjx/CS498_DL_Project/data/train/0/0/c/00cba0067c078490.jpg
/mnt/data/sjx/CS498_DL_Project/data/train/0/0/c/00cba0067c078490.jpg
/mnt/data/sjx/CS498_DL_Project/data/train/0/0/c/00cba0067c078490.jpg


(torch.Size([3, 672, 672]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 512, 512]))

# Model

In [33]:
class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine 

In [36]:
sigmoid = torch.nn.Sigmoid()
class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class Swish_module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)

    
 
    
class enet_arcface_FINAL(nn.Module):

    def __init__(self, enet_type, out_dim):
        super(enet_arcface_FINAL, self).__init__()
        self.enet = geffnet.create_model(enet_type.replace('-', '_'), pretrained=None)
        self.feat = nn.Linear(self.enet.classifier.in_features, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.classifier = nn.Identity()
 
    def forward(self, x):
        x = self.enet(x)
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)
    
    
from rexnetv1 import ReXNetV1
from resnest.torch import resnest101
project_path = '/mnt/data/sjx/CS498_DL_Project'
class rex20_arcface(nn.Module):

    def __init__(self, enet_type, out_dim, load_pretrained=False):
        super(rex20_arcface, self).__init__()
        self.enet = ReXNetV1(width_mult=2.0)
        if load_pretrained:
            pretrain_wts = os.path.join(project_path, '/Google-Landmark-Recognition-2020-3rd-Place-Solution/weights/rexnetv1_2.0x.pth')
            sd = torch.load(pretrain_wts)
            self.enet.load_state_dict(sd, strict=True)        
        
        self.feat = nn.Linear(self.enet.output[1].in_channels, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.output = nn.Identity()
    
    def forward(self, x):
        x = self.enet(x)
        if x.ndim==1: 
            x = x.unsqueeze(0)          
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)    
    
class nest101_arcface(nn.Module):

    def __init__(self, enet_type, out_dim):
        super(nest101_arcface, self).__init__()
        self.enet = resnest101(pretrained=False)
        self.feat = nn.Linear(self.enet.fc.in_features, 512)
        self.swish = Swish_module()
        self.metric_classify = ArcMarginProduct_subcenter(512, out_dim)
        self.enet.fc = nn.Identity()    
    def forward(self, x):
        x = self.enet(x)
        x = self.swish(self.feat(x))
        return F.normalize(x), self.metric_classify(x)    

In [37]:
def load_model(model, model_file):
    state_dict = torch.load(model_file)
    if "model_state_dict" in state_dict.keys():
        state_dict = state_dict["model_state_dict"]
    state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
#     del state_dict['metric_classify.weight']
    model.load_state_dict(state_dict, strict=True)
    print(f"loaded {model_file}")
    model.eval()    
    return model

In [ ]:
!ls -lrth ../input/b7ns-final-672-300w-f0-load13-load1-14ep/b7ns_final_672_300w_f0_load13_load1_14ep_fold0_ep4.pth
!ls -lrth ../input/b6ns-final-768-300w-f1-load28-5ep-1e-5/b6ns_final_768_300w_f1_load28_5ep_1e-5_fold1_ep5.pth
!ls -lrth ../input/b5ns-final-768-300w-f2-load16-20ep/b5ns_final_768_300w_f2_load16_20ep_fold2_ep1.pth
!ls -lrth ../input/b4ns-final-768-300w-f0-load16-20ep-load1-20ep/b4ns_final_768_300w_f0_load16_20ep_load1_20ep_fold0_ep4.pth
!ls -lrth ../input/b3ns-final-768-300w-f1-load29-5ep5ep/b3ns_final_768_300w_f1_load29_5ep5ep_fold1_ep5.pth
!ls -lrth ../input/nest101-final-768-300w-f4-load16-19ep-load1-16ep/nest101_final_768_300w_f4_load16_19ep_load1_16ep_fold4_ep5.pth
!ls -lrth ../input/rex20-ddp-final-768-300w-f4-35ep-load20resume/rex20_DDP_final_768_300w_f4_35ep_load20resume_fold4_ep31.pth
!ls -lrth ../input/b6ns-ddp-final-512-300w-f1-40ep/b6ns_DDP_final_512_300w_f1_40ep_fold1_ep36.pth
!ls -lrth ../input/b5ns-final-768-300w-f2-load33-5ep-3e-5-32g/b5ns_final_768_300w_f2_load33_5ep_3e-5_32G_fold2_ep4.pth

In [ ]:
model_b7 = enet_arcface_FINAL('tf_efficientnet_b7_ns', out_dim=out_dim).to(device)
model_b7 = load_model(model_b7, '../input/b7ns-final-672-300w-f0-load13-load1-14ep/b7ns_final_672_300w_f0_load13_load1_14ep_fold0_ep4.pth')

model_b6 = enet_arcface_FINAL('tf_efficientnet_b6_ns', out_dim=out_dim).to(device)
model_b6 = load_model(model_b6, '../input/b6ns-final-768-300w-f1-load28-5ep-1e-5/b6ns_final_768_300w_f1_load28_5ep_1e-5_fold1_ep5.pth')

model_b5 = enet_arcface_FINAL('tf_efficientnet_b5_ns', out_dim=out_dim).to(device)
model_b5 = load_model(model_b5, '../input/b5ns-final-768-300w-f2-load16-20ep/b5ns_final_768_300w_f2_load16_20ep_fold2_ep1.pth')

model_b4 = enet_arcface_FINAL('tf_efficientnet_b4_ns', out_dim=out_dim).to(device)
model_b4 = load_model(model_b4, '../input/b4ns-final-768-300w-f0-load16-20ep-load1-20ep/b4ns_final_768_300w_f0_load16_20ep_load1_20ep_fold0_ep4.pth')

model_b3 = enet_arcface_FINAL('tf_efficientnet_b3_ns', out_dim=out_dim).to(device)
model_b3 = load_model(model_b3, '../input/b3ns-final-768-300w-f1-load29-5ep5ep/b3ns_final_768_300w_f1_load29_5ep5ep_fold1_ep5.pth')

model_nest101 = nest101_arcface('nest101', out_dim=out_dim).to(device)
model_nest101 = load_model(model_nest101, '../input/nest101-final-768-300w-f4-load16-19ep-load1-16ep/nest101_final_768_300w_f4_load16_19ep_load1_16ep_fold4_ep5.pth')

model_rex2 = rex20_arcface('rex2.0', out_dim=out_dim).to(device)
model_rex2 = load_model(model_rex2, '../input/rex20-ddp-final-768-300w-f4-35ep-load20resume/rex20_DDP_final_768_300w_f4_35ep_load20resume_fold4_ep31.pth')

model_b6b = enet_arcface_FINAL('tf_efficientnet_b6_ns', out_dim=out_dim).to(device)
model_b6b = load_model(model_b6b, '../input/b6ns-ddp-final-512-300w-f1-40ep/b6ns_DDP_final_512_300w_f1_40ep_fold1_ep36.pth')

model_b5b = enet_arcface_FINAL('tf_efficientnet_b5_ns', out_dim=out_dim).to(device)
model_b5b = load_model(model_b5b, '../input/b5ns-final-768-300w-f2-load33-5ep-3e-5-32g/b5ns_final_768_300w_f2_load33_5ep_3e-5_32G_fold2_ep4.pth')

In [ ]:
!nvidia-smi

In [ ]:
with open(os.path.join(model_dir, 'idx2landmark_id.pkl'), 'rb') as fp:
    idx2landmark_id = pickle.load(fp)
    landmark_id2idx = {idx2landmark_id[idx]: idx for idx in idx2landmark_id.keys()}
    
pred_mask = pd.Series(df.landmark_id.unique()).map(landmark_id2idx).values

In [ ]:
TOP_K = 5
CLS_TOP_K = 5
if True:
    with torch.no_grad():
        feats = []
        for img0, img1,img3 in tqdm(query_loader): # 672, 768, 512
            img0 = img0.cuda()
            img1 = img1.cuda()
            img3 = img3.cuda()
            
            feat_b7,_      = model_b7(img0)
            feat_b6,_      = model_b6(img1)
            feat_b5,_      = model_b5(img1)
            feat_b4,_      = model_b4(img1)
            feat_b3,_      = model_b3(img1)            
            feat_nest101,_ = model_nest101(img1)
            feat_rex2,_    = model_rex2(img1)
            feat_b6b,_     = model_b6b(img3)
            feat_b5b,_     = model_b5b(img1)            
            feat = torch.cat([feat_b7,feat_b6,feat_b5,feat_b4,feat_b3,feat_nest101,feat_rex2,feat_b6b,feat_b5b],dim=1)            
#             print(feat.shape)
            feats.append(feat.detach().cpu())
        feats = torch.cat(feats)
        feats = feats.cuda()
        feat = F.normalize(feat)        

        PRODS = []
        PREDS = []
        PRODS_M = []
        PREDS_M = []        
        for img0, img1,img3 in tqdm(test_loader):
            img0 = img0.cuda()
            img1 = img1.cuda()
            img3 = img3.cuda()
            
            probs_m = torch.zeros([4, 81313],device=device)
            feat_b7,logits_m      = model_b7(img0); probs_m += logits_m
            feat_b6,logits_m      = model_b6(img1); probs_m += logits_m
            feat_b5,logits_m      = model_b5(img1); probs_m += logits_m
            feat_b4,logits_m      = model_b4(img1); probs_m += logits_m
            feat_b3,logits_m      = model_b3(img1) ; probs_m += logits_m
            feat_nest101,logits_m = model_nest101(img1); probs_m += logits_m
            feat_rex2,logits_m    = model_rex2(img1); probs_m += logits_m
            feat_b6b,logits_m     = model_b6b(img3); probs_m += logits_m
            feat_b5b,logits_m     = model_b5b(img1) ; probs_m += logits_m
            feat = torch.cat([feat_b7,feat_b6,feat_b5,feat_b4,feat_b3,feat_nest101,feat_rex2,feat_b6b,feat_b5b],dim=1)
            feat = F.normalize(feat)

            probs_m = probs_m/9
            probs_m[:, pred_mask] += 1.0
            probs_m -= 1.0              

            (values, indices) = torch.topk(probs_m, CLS_TOP_K, dim=1)
            probs_m = values
            preds_m = indices              
            PRODS_M.append(probs_m.detach().cpu())
            PREDS_M.append(preds_m.detach().cpu())            
            
            distance = feat.mm(feats.t())
            (values, indices) = torch.topk(distance, TOP_K, dim=1)
            probs = values
            preds = indices    
            PRODS.append(probs.detach().cpu())
            PREDS.append(preds.detach().cpu())

        PRODS = torch.cat(PRODS).numpy()
        PREDS = torch.cat(PREDS).numpy()
        PRODS_M = torch.cat(PRODS_M).numpy()
        PREDS_M = torch.cat(PREDS_M).numpy()


In [ ]:
!nvidia-smi

In [ ]:
# map both to landmark_id
gallery_landmark = df['landmark_id'].values
PREDS = gallery_landmark[PREDS]
PREDS_M = np.vectorize(idx2landmark_id.get)(PREDS_M)

In [ ]:
PREDS.min(), PREDS.max(), PREDS_M.min(), PREDS_M.max()

In [ ]:
PREDS[:3,:]

In [ ]:
PREDS_M[:3,:]

In [ ]:
PRODS[:3,:]

In [ ]:
PRODS_M[:3,:]

In [ ]:
PRODS_F = []
PREDS_F = []
for i in tqdm(range(PREDS.shape[0])):
    tmp = {}
    classify_dict = {PREDS_M[i,j] : PRODS_M[i,j] for j in range(CLS_TOP_K)}
    for k in range(TOP_K):
        lid = PREDS[i, k]
        tmp[lid] = tmp.get(lid, 0.) + float(PRODS[i, k]) ** 9 * classify_dict.get(lid,1e-8)**10
    pred, conf = max(tmp.items(), key=lambda x: x[1])
    PREDS_F.append(pred)
    PRODS_F.append(conf)

In [ ]:
PREDS_F[:10]

In [ ]:
PRODS_F[:10]

In [ ]:
df_test['pred_id'] = PREDS_F
df_test['pred_conf'] = PRODS_F

In [ ]:
df_sub['landmarks'] = df_test.apply(lambda row: f'{row["pred_id"]} {row["pred_conf"]}', axis=1)
df_sub.to_csv('submission.csv', index=False)

In [ ]:
df_sub.head()